### Imports

In [1]:
import numpy as np
from datetime import datetime
import os
import itertools as it
from env import *
import pandas as pd
import random
from Compact_Beurele import *
from Compact_MO import *

#### Loading Environment<br>
<B>Here:</B> tiger2 and discount=1  

In [2]:
e=tiger_POMDP_env(read_config=True,config_address='./tiger2.json',parameters=None)
num_to_act=dict(zip(list(e.actions.values()),list(e.actions.keys())))
e.discount_factor=1

{'state_change_r2l': 0.1, 'state_change_l2r': 0.1, 'false_observation_get_l_while_r': 0.1, 'false_observation_get_r_while_l': 0.2, 'reward_listen': -0.1, 'reward_low_incorrect': -0.2, 'reward_high_incorrect': -0.4, 'reward_low_correct': 0.2, 'reward_high_correct': 0.4, 'discount_factor': 1, 'initial_wealth': 0}


## Simulation parameters

In [26]:
max_depth=2                 # maximum depth of planning
planning_depth=max_depth    # maximum depth of planning

# For Three exponential terms: 
exp_vorfaktoren=[-1,0.1,2]  # Powers of exponential components
exp_weights=[-1,1,2]        # weights of exponential components

# Initial beliefs ( same for both methods)
initial_Mu=[0.5,0.5]        # initial beliefs about states. 
theta_0=initial_Mu           # Initial information vectors are same for all exponential terms

initial_observation=0       # initial observation before any action
initial_wealth=0            # initial amount of accumulated reward at the beginning of the simulation

# Beaurle method simulations

### Optimized Continious

set up and creats

In [4]:
###############    Setting Environment
e.reset()                             # reset the environment
initial_observation=e.current_state   # make the initial signal from environment same as what actual 
                                      #state it has 
observation=initial_observation       # record the observation
state=e.current_state                 # record the state

##############      Making Agent
# Create the agent
ag=Bauerle_Rieder_agent(environment=e ,max_time_step=max_depth)

value iteration

In [6]:
# Run its value iteration
b_S,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth,
    exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren)

simulation

In [7]:
# reset the agent 
ag.reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: (1, (0.5, 0.5), 0)

             action: open_right_high
             **
reward: 0.4 new observation: 1

new internal state: (1, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25), 1)

t= 1
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: (1, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25), 1)

             action: open_right_high
             **
reward: 0.4 new observation: 0

new internal state: (0, (0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125), 2)



#### Testing make Mu-points function 

<B>Making whole Mu-space and internal states</B><br>
These functions have application in discrete part, however in order to assess the size of state space without creating a "Discrete agent", they have written in a way that is accessible from Optimized_continious part as well.

Making Mu-points

In [15]:
mu_space,counts=ag.make_all_mu(partitioning_points_num=5,save_Mu=True, keep_all_steps=True)

In [17]:
counts

[6, 2002, 142506]

In [6]:
# Load pre-calculated points
f=ag.load_mu()

Making all internal_states

In [18]:
internal_states,counts=ag.make_all_internal_states_CN_discrete(partitioning_points_num=5,save_state_space=True, keep_all_steps=True)

In [22]:
counts

[12, 4004, 285012]

In [7]:
# Load pre-calculated states
f=ag.load_internal_states_CN_discrete()

### Discrete

set up and creats

In [11]:

###############    Setting Environment
e.reset()                             # reset the environment
initial_observation=e.current_state   # make the initial signal from environment same as what actual 
                                      #state it has 
observation=initial_observation       # record the observation
state=e.current_state                 # record the state
observations=e.observations.keys()    # fetch the observations
exp_num=len(exp_vorfaktoren)          # number of exponential terms
num_of_actions=len(e.actions)         # number of actions
num_of_observations=len(e.observations) #number of observations 

################ Making Agent

############################## Discrete agent variables 
max_time_step=max_depth     # depth of planning
r_round=5                   # number of decimal digits for rounding
partitioning_points_num=24  # number of partitioning points: determines precision of belief estimation

# creating the agent: ag2
ag2=Bauerle_Rieder_agent(environment=e ,max_time_step=max_depth)

value iteration

In [12]:
# Run its value iteration
b_optimized_mu_size,b_value_func,b_action_func,b_q_func=ag2.discrete_value_iteration(partitioning_points_num,initial_wealth,exp_weights,exp_vorfaktoren,rounding_prec_coeff=100000,
                                 r_round=5,save_reachables=True,save_transition_kernel=True,use_buffer=True,
                                 memory_threshold=1000000,keep_q_func=True)

simulation

In [14]:
# reset the agent 
ag2.reset(initial_mu_state=initial_Mu,initial_observation=initial_observation,initial_wealth=initial_wealth)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag2.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag2.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag2.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag2.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: (0, (0.5, 0.5), 0)

             action: open_right_high
             **
reward: -0.4 new observation: 0

new internal state: (0, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25), 1)

t= 1
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: (0, (0.25, 0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.25), 1)

             action: open_right_high
             **
reward: -0.4 new observation: 0

new internal state: (0, (0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125), 2)



# Multi-Variate (MO) method simulations

### Optimized Continious

set up and creats

In [27]:
###############    Setting Environment
e.reset()                             # reset the environment
initial_observation=e.current_state   # make the initial signal from environment same as what actual 
                                      #state it has 
observation=initial_observation       # record the observation
state=e.current_state                 # record the state



##############      Making Agent
# Create the agent
ag=Multi_Variate_agent(environment=e ,max_time_step=max_depth)


value iteration

In [28]:
# Run its value iteration
_,b_reachables,b_map,b_q_func,b_value_function,b_action_func=ag.continious_optimized_planning(initial_theta=theta_0,initial_observation=initial_observation,initial_wealth=initial_wealth,
    exp_weights=exp_weights,exp_vorfaktoren=exp_vorfaktoren,x_round=5,r_round=5)

simulation

In [29]:
# reset the agent 
ag.reset(theta_0=theta_0,initial_observation=initial_observation,initial_wealth=initial_wealth)

#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 0 tiger_right
last_observation: 0
-------------------------------

current internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0), (0, 0, 0), 0)

             action: open_right_high
             **
reward: -0.4 new observation: 1

new internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1), (-0.07795, 0.008, 0.14538), 1)

t= 1
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1), (-0.07795, 0.008, 0.14538), 1)

             action: open_right_high
             **
reward: 0.4 new observation: 1

new internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1), (-0.1559, 0.016, 0.29076), 2)



#### Testing make internal states function 

<B>Making whole internal state</B><br>
This functions has application in discrete part, however in order to assess the size of state space without creating a "Discrete agent", It has written in a way that is accessible from Optimized_continious part as well.

In [124]:
ag=Multi_Variate_agent(environment=e ,max_time_step=max_depth)
internal_states,counts=ag.make_all_internal_states(max_depth,10,initial_wealth,exp_weights,exp_vorfaktoren,rounding_prec_coeff=100000,r_round=3,save_space=True, keep_all_steps=True)

0
1
2


In [5]:
# test loading func
internal_state=ag.load_space()

### Discrete MO

set up and creats

In [30]:
###############    Setting Environment
e.reset()                             # reset the environment
initial_observation=e.current_state   # make the initial signal from environment same as what actual 
                                      #state it has 
observation=initial_observation       # record the observation
state=e.current_state                 # record the state
observations=e.observations.keys()    # fetch the observations
exp_num=len(exp_vorfaktoren)          # number of exponential terms
num_of_actions=len(e.actions)         # number of actions
num_of_observations=len(e.observations) #number of observations 

################ Making Agent

############################## Discrete agent variables 
max_time_step=max_depth     # depth of planning
r_round=5                   # number of decimal digits for rounding
partitioning_points_num=10  # number of partitioning points: determines precision of belief estimation

# Create the agent
ag2=Multi_Variate_agent(environment=e ,max_time_step=max_time_step)



value iteration

In [31]:
# Run its value iteration
m_space_size,m_value_func,m_action_func,m_q_func=ag2.discrete_value_iteration(partitioning_points_num,initial_wealth,exp_weights,exp_vorfaktoren,rounding_prec_coeff=100000,
                                 r_round=5,save_reachables=True,save_transition_kernel=True,use_buffer=True,
                                 memory_threshold=1000000,keep_q_func=True)

simulation

In [32]:
# reset the agent 
ag2.reset(theta_0=theta_0,initial_observation=initial_observation,initial_wealth=initial_wealth)


#############         Simulation

for t in range(planning_depth):
    
    print('t=',t)
    print('------')
    print('state:',state,e.states[state])
    print('last_observation:',observation)
    print('-------------------------------')
    print('')
    print('current internal state:',ag2.current_internal_state)
    print('')
    
    # agent select the action
    action,value_of_action,_=ag2.do_action()
    
    #environment feedback
    t1,t2,state,reward,observation=e.step(num_to_act[action])
    
    # agent update
    new_x=ag2.update_agent(new_observation=observation) 
    
    print('             action:',num_to_act[action])  
    print ('             **' )
    print('reward:',reward,'new observation:',observation)
    print('')
    print('new internal state:', ag2.current_internal_state)
    print('')
    print('============================')

t= 0
------
state: 1 tiger_left
last_observation: 1
-------------------------------

current internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1), (0, 0, 0), 0)

             action: open_right_high
             **
reward: 0.4 new observation: 1

new internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5), (-0.07795, 0.008, 0.14538), 1)

t= 1
------
state: 0 tiger_right
last_observation: 1
-------------------------------

current internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5), (-0.07795, 0.008, 0.14538), 1)

             action: open_right_high
             **
reward: -0.4 new observation: 0

new internal state: ((0.5, 0.5, 0.5, 0.5, 0.5, 0.5), (-0.1559, 0.016, 0.29076), 0)

